# HW2

Jaee Oh / Tejas Phanse / Mandadi Ayush Reddy

## Part A: DTM

In [48]:
import nltk
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import re
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from bs4 import BeautifulSoup
import nltk

nltk.download('stopwords')
nltk.download('wordnet') 
print("Libraries loaded successfully.")

[nltk_data] Downloading package stopwords to /home/jaee/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /home/jaee/nltk_data...


Libraries loaded successfully.


- Task1: Text Prerpcessing

In [49]:
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

# List of common words to be removed from full_text_clean column
common_word_list = {'followers', 'instagram', 'facebook', 'like', 'messages', 'support',
                    'sitemap', 'free', 'faq', 'twitter', 'search', 'subscribe', 'youtube', 
                    'feedback', 'linkedin', 'demo', 'menu', 'following', 'unsubscribe', 
                    'snapchat', 'member', 'tiktok', 'whatsapp', 
                    'profile', 'share', 'reddit', 'download', 'settings', 'notifications', 
                    'home', 'qa', 'people'}
stop_words = stop_words.union(common_word_list) # Add common words to NLTK stop words

# Common phrases to remove
phrase_list = ["find out more", "join now","terms & conditions", "terms and conditions","all rights reserved","free trial", 
             "click here", "about us", "contact us","terms of service", "privacy policy", "help center", "our story","our team",
             "read more", "learn more", "get started", "download now", "sign up", "log in"]

def preprocess(text):
    # Remove \n, \r, \t
    text = re.sub(r'[\n\r\t]+', ' ', text)

    # Remove HTML tags like <body>, but first remove script/style content
    soup = BeautifulSoup(text, 'html.parser')
    for tag in soup(['script', 'style', 'noscript']):
        tag.decompose()
    text = soup.get_text(separator=' ')
    
    # Lowercase
    text = text.lower()
    
    # Remove URLs
    text = re.sub(r'(?:https?://|ftp://|www\.)\S+|mailto:\S+|tel:\S+', '', text)
    
    # Remove emails
    text = re.sub(r'\b[a-z0-9._%+-]+@[a-z0-9.-]+\.[a-z]{2,}\b', '', text)
    
    # Remove phrases
    for phrase in phrase_list:
        text = text.replace(phrase, ' ')

    # Remove non-ASCII characters
    text = re.sub(r'[^\x00-\x7F]', ' ', text)
    
    # Keep only letters and spaces
    text = re.sub(r'[^a-z\s]', '', text)

    # Tokenize and filter with stop words and lemmatization
    tokens = text.split()
    tokens = [lemmatizer.lemmatize(t) for t in tokens if len(t) > 2 and t not in stop_words]
    
    return ' '.join(tokens)

In [50]:
# Load documents
documents = pd.read_csv('../data/dataset_for_BOW_Class.csv')
documents['text_clean'] = documents['full_text'].apply(preprocess)
print(documents['text_clean'].head())

# Basic CountVectorizer
count_vec = CountVectorizer()
count_matrix = count_vec.fit_transform(documents['text_clean'])  # Fixed

print(f"Matrix shape: {count_matrix.shape}")
print(f"  - {count_matrix.shape[0]} documents")
print(f"  - {count_matrix.shape[1]} unique terms in vocabulary")

# Be careful with large vocabularies - this could print thousands of words
vocab = count_vec.get_feature_names_out()
print(f"\nFirst 20 terms: {vocab[:20]}")
print(f"Last 20 terms: {vocab[-20:]}")

0    velvet mousse blush cream ebelebelle top page ...
1    forge carter cfcc stone inlay david douglas di...
2    travel news guide destination tip close update...
3    find ridgerun solution ridgerun developer wiki...
4    shooting ktvu fox live news weather sport cont...
Name: text_clean, dtype: object
Matrix shape: (25, 3216)
  - 25 documents
  - 3216 unique terms in vocabulary

First 20 terms: ['abdicate' 'abduction' 'abdulaziz' 'abdullah' 'abiding' 'abingdon' 'able'
 'abode' 'abounding' 'aboveboard' 'abril' 'abrupt' 'absolute' 'absolve'
 'abstraction' 'academy' 'accept' 'acceptance' 'access' 'accessability']
Last 20 terms: ['xact' 'xcelerator' 'xilinx' 'xlive' 'xshare' 'yanagita' 'year' 'yellow'
 'yet' 'yieldenhancer' 'yocto' 'york' 'yorkshire' 'youre' 'youve' 'yuan'
 'zebrafish' 'zfa' 'zip' 'zone']
